In [ ]:
import os, sys
sys.path.append(os.path.abspath("../../../"))
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np

# Config dict to set the logging level
import logging.config
DEFAULT_LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'loggers': {
        '': {
            'level': 'WARN',
        },
    }
}

logging.config.dictConfig(DEFAULT_LOGGING)
# Disabling warnings output
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [ ]:
url = 'https://github.com/nlapier2/PySensemakr/blob/main/sensemakr/data/darfur.csv?raw=true'
df = pd.read_csv(url)
df.head()

In [ ]:
causal_graph = """dag {
age [pos="-0.197,-0.949"]
directlyharmed [pos="-0.165,0.301"]
farmer_dar [pos="-0.838,-0.982"]
female [pos="-1.841,-0.220"]
herder_dar [pos="0.710,-0.604"]
hhsize_darfur [pos="-2.044,0.164"]
pastvoted [pos="1.201,-0.226"]
peacefactor [outcome,pos="-0.048,0.900"]
village [pos="1.831,0.269"]
directlyharmed -> peacefactor
farmer_dar -> directlyharmed
female -> directlyharmed
herder_dar -> directlyharmed
hhsize_darfur -> directlyharmed
pastvoted -> directlyharmed
village -> directlyharmed
age -> pastvoted
female -> peacefactor
age -> peacefactor
pastvoted -> peacefactor
village -> peacefactor
farmer_dar -> peacefactor
herder_dar -> peacefactor
hhsize_darfur -> peacefactor
}
"""

In [ ]:
model= dowhy.CausalModel(
        data = df,
        graph=causal_graph,
        treatment="directlyharmed",
        outcome='peacefactor')
model.view_model()
from IPython.display import Image, display
display(Image(filename="causal_model.png"))

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
estimate = model.estimate_effect(identified_estimand,method_name="backdoor.linear_regression")

In [ ]:
refute = model.refute_estimate(identified_estimand, estimate , method_name = "add_unobserved_common_cause",
                              simulated_method_name = "Carlos", benchmark_covariates = ["female"],
                              kd = [1,2,3], 
                              formula='peacefactor ~ directlyharmed + age + farmer_dar + herder_dar + pastvoted + hhsize_darfur + female + village')

In [ ]:
refute.stats

In [ ]:
refute.bounds_result